In [1]:
import sys
import os
# Get the parent directory
parent_dir = os.getcwd()
# Add the parent directory to sys.path
sys.path.append(parent_dir)

In [2]:
import numpy as np
from itertools import combinations
import scipy
import qokit
from qokit import get_qaoa_objective
from qokit.parameter_utils import parallel_execution, initialize_csv, generate_p_values, interpolation_basis, fine_tune_result, save_result, log_walltime
from contextlib import contextmanager
import time
from typing import Optional, Tuple, Callable

In [4]:
# Constants for the linear ramp initialization
GAMMA_START = 0.0
GAMMA_END = 0.1
BETA_START = -0.1
BETA_END = 0.0


In [5]:
# Constants
max_evals = 40000
ratio_threshold = 0.98
overlap_threshold = 0.5
rhobeg = 0.01

In [6]:
def initialize_parameters(gamma: Optional[np.ndarray], beta: Optional[np.ndarray], p: int) -> Tuple[np.ndarray, np.ndarray]:
    if gamma is None and beta is None:
        gamma = np.linspace(GAMMA_START, GAMMA_END, p)
        beta = np.linspace(BETA_START, BETA_END, p)
    elif gamma is not None and beta is not None:
        assert len(gamma) == len(beta)
    return gamma, beta

In [7]:
@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

In [8]:
def II_schedule(
    f_overlap: Callable[[np.ndarray], float],
    f: Callable[[np.ndarray], float],
    gs_energy: float,
    N: int,
    pmax: int = 100,
    step: int = 5,
    epsilon: float = 0.01,
    patience: int = 5,
    filename: str = None,
    p0: int = 5,
    gamma: Optional[np.ndarray] = None,
    beta: Optional[np.ndarray] = None,
    save: bool = True,
    basis: str = 'chebyshev',
    optimizer: str = 'bobyqa',
    maxiter: int = 1000,
    overlap_conv: bool = True,
    max_energy: float = 0,
    ratio_threshold: float = 0.98,
    rhobeg: float = 0.01
) -> dict:
    start_time = time.time()
    total_eval = 0
    
    if filename is None:
        filename = f'II_N_{N}_pmax_{pmax}_step_{step}_optimizer_{optimizer}.csv'

    if save and filename and not os.path.exists(filename):
        initialize_csv(filename)

    gamma, beta = initialize_parameters(gamma, beta, p=p0)

    if p0 < 5:
        init_p = np.array([1,2, 3, 4])
        p_values = np.concatenate((init_p, generate_p_values(5, pmax, step)))
        
    else:
        p_values = generate_p_values(p0, pmax, step)
        
    num_coeffs = 5
    
    counter = 0
    prev_approx_ratio = 0
    print(f"Optimal value for N={N} is {gs_energy}")

    for p in p_values:
        if p<=5:
            num_coeffs = p
        
        gamma, beta = interpolation_basis(gamma, beta, p, num_coeffs, basis)
        result = fine_tune_result(f_overlap, f, gs_energy, N, p, num_coeffs, gamma, beta, basis, optimizer, rhobeg, maxiter, max_energy)
        gamma, beta = result['gamma'], result['beta']
        approx_ratio = result['approx_ratio']
        overlap = result['overlap']
        total_eval += result['evaluations']

        if save and filename:
            save_result(result, filename)

        if overlap_conv==True:
            if overlap > overlap_threshold or total_eval > max_evals:
                break
                
        if overlap_conv==False:
            if approx_ratio > ratio_threshold or total_eval > max_evals:
                break

        improvement = (approx_ratio - prev_approx_ratio) / approx_ratio
        if improvement < epsilon:
            counter += 1

        if counter == patience:
            num_coeffs += 1
            counter = 0

        prev_approx_ratio = approx_ratio

    log_walltime(start_time)
    print(f"Total evaluations: {total_eval}")
    return result

In [9]:
def gs(N, terms):
    simclass = qokit.fur.choose_simulator(name='auto')
    sim = simclass(N, terms=terms)
    sorted_diag = np.sort(sim.get_cost_diagonal())
    gs = sorted_diag[0]
    return gs

In [10]:
def createList(r1, r2):
    return [item for item in range(r1, r2+1)]

In [11]:
def run_trial(N, trial, base_seed, pmax, step, optimizer, dir_name):
    with suppress_stdout():
        seed = base_seed + trial
        np.random.seed(seed)
        terms = [(np.random.normal(), spin_pair) for spin_pair in combinations(range(N), r=2)]
        
        f_overlap = get_qaoa_objective(N, terms=terms, objective="overlap")
        f = get_qaoa_objective(N, terms=terms, objective="expectation")
        
        gs_energy = gs(N, terms)
        
        filename = os.path.join(dir_name, f"II_N_{N}_SK_results_seed_{seed}_pmax_{pmax}_step_{step}_optimizer_{optimizer}.csv")

        result = II_schedule(
            f_overlap=f_overlap,
            f=f,
            gs_energy=gs_energy,
            N=N,
            pmax=pmax,
            step=step,
            save=True,
            filename=filename,
            p0=5,
            overlap_conv=True
        )

In [12]:
def main(N):
    base_seed = 0
    num_trials = 5
    pmax = 100
    step = 5
    optimizer = 'bobyqa'
    num_gpus = 4
    dir_name = f"ii_results/SK_N{N}"
    os.makedirs(dir_name, exist_ok=True)
    
    inputs = [(N, trial, base_seed, pmax, step, optimizer, dir_name) for trial in range(num_trials)]
    
    results = parallel_execution(run_trial, inputs, num_gpus)

In [13]:
if __name__ == "__main__":    
    N = 6
    main(N)